# Gaussian elimination and LU factorization

This notebook presents calculations of Gauss elimination and LU factorization methods for matrix transformation and equation solving.

In [1]:
import numpy as np

# My birthdate is 07.17
birthday_matrix_dim = 7 + 17

def fill_matrix_with_rand(n, low=-1000, high=1000):
    return np.random.randint(low, high + 1, size=(n, n))

matrix = fill_matrix_with_rand(birthday_matrix_dim)

# Since the birthday matrix is quite big, for method sdemonstration purposes a smaller matrix will be used
demonstartion_matrix = fill_matrix_with_rand(8)

print("Demonstartion matrix:\n" , demonstartion_matrix )


Demonstartion matrix:
 [[-162 -704 -728 -353  743  780  -87 -635]
 [ 809 -826  939 -779   78 -339  451 -971]
 [ 162 -887  849 -337  770  980  -55 -538]
 [ 375 -111  -49  462  402  616 -741  546]
 [ -85 -694 -283  817  845   14 -620  -71]
 [ 562  847  707  105  177  238  190  -67]
 [ 377 -635 -156 -635 -579 -599 -838 -503]
 [ 525 -916  916  878 -847  581 -290 -982]]


### Gaussian Elimination without Pivoting

Gaussian elimination is used to solve systems of linear equations by transforming the augmented matrix into row echelon form. It involves eliminating variables from equations, row by row, to eventually obtain an upper triangular matrix. By ensuring that the leading coefficient in each row is 1, the process simplifies the back substitution operation used to find variable values.

In [2]:
def gauss_elimination_without_pivoting(input_matrix):
    A = input_matrix
    n = len(A)
    for i in range(n):
        if A[i][i] == 0:
            raise ValueError("Zero division error. Pivot element is zero.")
        for j in range(i+1, n):
            factor = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]
    return A

gauss_elim = gauss_elimination_without_pivoting(demonstartion_matrix)
print(demonstartion_matrix)

[[ -162  -704  -728  -353   743   780   -87  -635]
 [    0 -4341 -2696 -2541  3788  3556    16 -4142]
 [    0     0  1109   241   124   456  -147   345]
 [    0     0     0   804   675  1263 -1034   940]
 [    0     0     0     0  -807 -2550   913  -839]
 [    0     0     0     0     0 -2624  1418 -1998]
 [    0     0     0     0     0     0  -931  -361]
 [    0     0     0     0     0     0     0 -4464]]


### Gaussian Elimination with Pivoting

Similar to Gaussian elimination without pivoting, but with an added pivoting step to avoid division by zero and reduce numerical errors. Pivoting involves swapping rows so that the largest coefficient (pivot) is placed at the current position, ensuring the stability and accuracy of the algorithm.

In [3]:
def gauss_elimination_with_pivoting(input_matrix):
    A = input_matrix
    n = len(A)
    for i in range(n):
        max_index = i
        for j in range(i+1, n):
            if abs(A[j][i]) > abs(A[max_index][i]):
                max_index = j
        A[[i, max_index]] = A[[max_index, i]]  # Zamiana wierszy
        if A[i][i] == 0:
            raise ValueError("Zero division error. Pivot element is zero.")
        for j in range(i+1, n):
            factor = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= factor * A[i][k]
    return A

gauss_elim_pivot = gauss_elimination_with_pivoting(demonstartion_matrix)
print("Gauss Elimination with Pivoting Result == Gauss Elimination without Pivoting Result ? ", (gauss_elim_pivot == gauss_elim).all())


Gauss Elimination with Pivoting Result == Gauss Elimination without Pivoting Result ?  True


### LU Factorization without Pivoting

LU (Lower-Upper) factorization decomposes a square matrix into the product of a lower triangular matrix and an upper triangular matrix. This method is useful for solving systems of linear equations and finding matrix inverses. Without pivoting, the matrices L and U are directly constructed based on the original matrix A.

In [5]:
def lu_factorization_without_pivoting(A):
    n = len(A)
    L = np.eye(n)
    U = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            U[i][j] = A[i][j] - sum(L[i][k] * U[k][j] for k in range(i))
        for j in range(i+1, n):
            L[j][i] = (A[j][i] - sum(L[j][k] * U[k][i] for k in range(i))) / U[i][i]
    return L, U

L, U = lu_factorization_without_pivoting(demonstartion_matrix)
print("Matrix L:")
print(L)
print("\nMatrix U:")
print(U)

Matrix L:
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [-0.  1.  0.  0.  0.  0.  0.  0.]
 [-0. -0.  1.  0.  0.  0.  0.  0.]
 [-0. -0.  0.  1.  0.  0.  0.  0.]
 [-0. -0.  0.  0.  1.  0.  0.  0.]
 [-0. -0.  0.  0. -0.  1.  0.  0.]
 [-0. -0.  0.  0. -0. -0.  1.  0.]
 [-0. -0.  0.  0. -0. -0. -0.  1.]]

Matrix U:
[[ -162.  -704.  -728.  -353.   743.   780.   -87.  -635.]
 [    0. -4341. -2696. -2541.  3788.  3556.    16. -4142.]
 [    0.     0.  1109.   241.   124.   456.  -147.   345.]
 [    0.     0.     0.   804.   675.  1263. -1034.   940.]
 [    0.     0.     0.     0.  -807. -2550.   913.  -839.]
 [    0.     0.     0.     0.     0. -2624.  1418. -1998.]
 [    0.     0.     0.     0.     0.     0.  -931.  -361.]
 [    0.     0.     0.     0.     0.     0.     0. -4464.]]


### LU Factorization with Pivoting

An enhanced version of LU factorization that incorporates pivoting to improve stability and accuracy. It involves row swapping for pivoting and reordering rows to ensure that pivot elements are the largest in absolute value. This method is particularly effective when working with ill-conditioned matrices or matrices with small pivot elements.

In [6]:
def lu_factorization_with_pivoting(A):
    n = len(A)
    L = np.eye(n)
    U = np.zeros((n, n))
    P = np.eye(n)
    for i in range(n):
        max_index = i
        for j in range(i+1, n):
            if abs(A[j][i]) > abs(A[max_index][i]):
                max_index = j
        A[[i, max_index]] = A[[max_index, i]]  # Zamiana wierszy
        P[[i, max_index]] = P[[max_index, i]]  # Zamiana wierszy w macierzy permutacji
        for j in range(i, n):
            U[i][j] = A[i][j] - sum(L[i][k] * U[k][j] for k in range(i))
        for j in range(i+1, n):
            L[j][i] = (A[j][i] - sum(L[j][k] * U[k][i] for k in range(i))) / U[i][i]
    return P, L, U


P, L, U = lu_factorization_with_pivoting(demonstartion_matrix)
print("Permutation matrix P:")
print(P)
print("\nMatrix L:")
print(L)
print("\nMatrix U:")
print(U)


Permutation matrix P:
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

Matrix L:
[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [-0.  1.  0.  0.  0.  0.  0.  0.]
 [-0. -0.  1.  0.  0.  0.  0.  0.]
 [-0. -0.  0.  1.  0.  0.  0.  0.]
 [-0. -0.  0.  0.  1.  0.  0.  0.]
 [-0. -0.  0.  0. -0.  1.  0.  0.]
 [-0. -0.  0.  0. -0. -0.  1.  0.]
 [-0. -0.  0.  0. -0. -0. -0.  1.]]

Matrix U:
[[ -162.  -704.  -728.  -353.   743.   780.   -87.  -635.]
 [    0. -4341. -2696. -2541.  3788.  3556.    16. -4142.]
 [    0.     0.  1109.   241.   124.   456.  -147.   345.]
 [    0.     0.     0.   804.   675.  1263. -1034.   940.]
 [    0.     0.     0.     0.  -807. -2550.   913.  -839.]
 [    0.     0.     0.     0.     0. -2624.  1418. -1998.]
 [    0.     0.     0.     0.     0.     0.  -931.  -361.]
 [    0.     0.     0.     0.     0.     0.  

## Solving system of equations

In [7]:
def gauss_solve(triagnle_matrix):
    A = triagnle_matrix
    n = len(A)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = A[i][-1] / A[i][i]
        for j in range(i-1, -1, -1):
            A[j][-1] -= A[j][i] * x[i]
    return x


triangle_matrix = gauss_elimination_without_pivoting(matrix)
print("Solving with Gauss no pivoting:\n", gauss_solve(triangle_matrix))
triangle_matrix_pivot = gauss_elimination_with_pivoting(matrix)
print("Solving with Gauss with pivoting:\n", gauss_solve(triangle_matrix_pivot))

Solving with Gauss no pivoting:
 [-0. -0. -0.  0. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.
  0.  0. -0. -0.  0.  1.]
Solving with Gauss with pivoting:
 [-0. -0. -0.  0. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0. -0. -0.  0.
  0.  0. -0. -0.  0.  1.]


In [75]:
import numpy as np

def lu_factorization_solve(A, b, L, U):
    """
    A (numpy.ndarray): Coefficient matrix.
    b (numpy.ndarray): Right-hand side vector.
    """
    n = len(A)
    # Solve Ly = b
    y = np.linalg.solve(L, b)

    # Solve Ux = y
    x = np.linalg.solve(U, y)

    return x

L, U = lu_factorization_without_pivoting(matrix)
A = matrix[:-1]
b = matrix[-1]
print("Solving with LU no pivoting:\n", lu_factorization_solve(A, b, L, U))

_, L_pivot, U_pivot = lu_factorization_with_pivoting(matrix)
A_pivot = matrix[:-1]
b_pivot = matrix[-1]
print("Solving with LU with pivoting:\n", lu_factorization_solve(A, b, L, U))

Solving with LU no pivoting:
 [-0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0.  0.  0.
  0. -0.  0.  0. -0.  1.]
Solving with LU with pivoting:
 [-0. -0.  0.  0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0. -0.  0.  0.  0.
  0. -0.  0.  0. -0.  1.]
